# 📚 GitManager — Интерактивное руководство

## 🗂 Содержание
1. [Описание программы](#1-описание-программы)
2. [Архитектура (из ARCHITECTURE.md)](#2-архитектура)
3. [Функционал](#3-функционал)
4. [Запуск .venv](#4-запуск-venv)
5. [Запуск GUI из Jupyter](#5-запуск-gui-из-jupyter)
6. [Тесты](#6-тесты)
7. [Скриншоты](#7-скриншоты)

## 1. Описание программы

**GitManager** — PyQt6-приложение для управления Git без терминала.

- Цель: упростить начало работы с Git
- Аудитория: новички, преподаватели, автоматизаторы
- Особенности:
  - Инициализация репозитория
  - Коммит и push с GUI
  - Централизованное логирование
  - Сохранение конфигурации

In [1]:
# Чтение и отображение ARCHITECTURE.md
from IPython.display import Markdown, display
from pathlib import Path

arch_path = Path("docs/diagrams") / "ARCHITECTURE.md"
if arch_path.exists():
    content = arch_path.read_text(encoding='utf-8')
    display(Markdown(content))
else:
    print("❌ Файл ARCHITECTURE.md не найден. Убедитесь, что он в docs/")


# 🏗️ Архитектура GitManager

Документ описывает структуру и принципы работы приложения `GitManager` — графического инструмента для управления Git без знания терминала.

---

## 🧩 Общая архитектура

`GitManager` построен по принципу **чистой архитектуры (Clean Architecture)** с чётким разделением ответственности:

- +------------------+
- | GUI | ← Представление (View)
- +------------------+
- ↓
- +------------------+
- | Core / API | ← Бизнес-логика (Controller)
- +------------------+
- ↓
- +------------------+
- | Commands | ← Git-команды как данные
- +------------------+
- ↓
- +------------------+
- | subprocess | ← Внешняя система (Git CLI)
- +------------------+

---
## 📁 Структура проекта
- git_manager/
- ├── app.py ← Точка входа, инициализация
- ├── core/ ← Бизнес-логика
- │ ├── project_manager.py ← Управление репозиторием
- │ └── git_executor.py ← Выполнение команд
- ├── commands/ ← Определение Git-команд
- │ ├── init.py
- │ ├── status.py
- │ ├── add.py
- │ ├── commit.py
- │ ├── push.py
- │ └── remote.py
- ├── gui/ ← Представление
- │ ├── main_window.py ← Главное окно
- │ ├── settings_dialog.py ← Настройки
- │ └── icons/ ← Ресурсы
- ├── utils/ ← Общие утилиты
- │ ├── logger.py ← Единый логгер
- │ └── file_handler.py ← Работа с JSON
- ├── config/ ← Конфигурация (runtime)
- ├── logs/ ← Логи (runtime)
- └── docs/ ← Документация
- ├── ARCHITECTURE.md ← Этот файл
- └── diagrams/ ← Диаграммы

---

## 🧠 Ключевые принципы

### 1. **Единая точка инициализации**
- Логгер создаётся **только в `app.py`**
- Передаётся по цепочке в `MainWindow` → `ProjectManager`
- Исключает дублирование хендлеров

### 2. **Команды как данные**
- Каждая Git-команда — функция, возвращающая `(cmd, cwd)`
- Позволяет тестировать логику без вызова `subprocess`

### 3. **Иммутабельность конфигурации**
- Конфиг (`config/user_config.json`) загружается один раз при старте
- Изменяется только через `SettingsDialog`
- Сохраняется через `FileHandler.save_json`

### 4. **Безопасность GUI**
- `subprocess` запускается с `startupinfo` — **окно консоли не появляется**
- Все пути нормализуются и валидируются
- Игнорируемые файлы (`__pycache__`, `.pyc`) фильтруются

---

## 🔗 Поток управления

1. Пользователь выбирает папку проекта
2. `MainWindow` создаёт `ProjectManager(project_dir, logger)`
3. При действии (например, "Commit") вызывается:
   - `ProjectManager.commit_files(files, msg)`
   - → `git add` + `git commit`
4. Результат логируется и отображается в GUI
5. При `push`:
   - Сначала `git pull` (если нужно)
   - Затем `git push`

---

## 📦 Зависимости

- **PyQt6** — GUI
- **Python 3.10+** — основной интерпретатор
- **Git CLI** — внешняя зависимость (должна быть в `PATH`)

> ⚠️ Приложение **не заменяет Git**, а **является обёрткой** вокруг `git` команд.

---

## 🧪 Тестируемость

- `ProjectManager` можно тестировать без GUI
- `commands/*.py` — чистые функции, легко мокать
- Логгер можно заменить на `MockLogger` в тестах

---

## 📈 Планы на будущее

- Поддержка веток (`git branch`, `git checkout`)
- Интеграция с `git stash`
- Поддержка SSH-ключей
- Автообновление
- Поддержка `git clone`

In [2]:
# Отображение CHANGELOG.md
from pathlib import Path
from IPython.display import Markdown, display

changelog_path = Path("docs/releases") / "CHANGELOG.md"
if changelog_path.exists():
    content = changelog_path.read_text(encoding='utf-8')
    display(Markdown(content))
else:
    print("❌ CHANGELOG.md не найден")

# 📜 История изменений: GitManager

## [v0.2] — 2025-08-20
### 🚀 Основные улучшения
- Добавлена **инициализация репозитория с привязкой к удалённому URL**
- Реализован **автосохранённый конфиг** (`config/user_config.json`)
- Добавлена **поддержка привязки к GitHub через ввод ссылки**
- Внедрено **централизованное логирование** через `utils/logger.py`
- Логи пишутся в `logs/system.log` с ротацией (до 5 файлов по 10 МБ)

### 🛠 Архитектурные изменения
- Проект разделён на уровни:
  - `core/` — логика и управление Git
  - `commands/` — чистые команды
  - `gui/` — интерфейс
  - `utils/` — общие инструменты
- Вместо дублирующих функций — использован `utils/file_handler.FileHandler`
- Реализован `ProjectManager` — единая точка управления репозиторием

### 🏗️ Архитектурные улучшения
- **Логгер инициализируется в одном месте (`app.py`) и передаётся по цепочке** — устранено дублирование и гарантируется согласованность
- Удалена прямая инициализация `get_logger()` в модулях — теперь все компоненты используют единый экземпляр
- Версия приложения (`v0.2`) вшита в логи через параметр, а не через формат

### 🐛 Исправлено
- Исправлено **появление консоли при коммите** (через `startupinfo`)
- Добавлена фильтрация игнорируемых файлов (например, `__pycache__/`, `.pyc`)
- Исправлено **неправильное отображение новых файлов** (`??` в `git status`)
- Добавлен `git pull` перед `push` для предотвращения `rejected`

### 🧩 Новые возможности
- При запуске **автозаполняется последний URL репозитория**
- Все ошибки и действия логируются с уровнями: `INFO`, `WARNING`, `ERROR`
- Поддержка `--windowed` в `.exe` без всплывающей консоли
- Готов к сборке в один `.exe` через PyInstaller

---

## [v0.1] — 2025-08-15
### 🌱 Первый релиз
- Создан GUI для управления Git
- Поддержка:
  - `git init`
  - `git add`
  - `git commit`
  - `git push`
- Визуализация изменённых файлов
- Ввод сообщения коммита
- Простой интерфейс с вкладками

> ⚠️ На этом этапе:
> - Не было логирования
> - Не было сохранения настроек
> - Были проблемы с `--windowed`
> - Не было поддержки `git pull`
> - Не было фильтрации `.gitignore`

---

## 📌 План на v0.3
- Поддержка веток (`git checkout`, `git branch`)
- Интеграция с `git stash`


In [3]:
# Запуск GitManager
import sys
import os
sys.path.append(os.getcwd())

from PyQt6.QtWidgets import QApplication
from GUI import MainWindow
from utils import get_logger

# Только если QApplication ещё не запущен
app = QApplication.instance()
if app is None:
    app = QApplication([])

logger = get_logger("Jupyter_tests")
window = MainWindow(logger=logger)
window.show()

# Чтобы не блокировать ячейку
print("✅ Окно запущено. Не закрывай этот блок, пока используешь GUI.")
app.exec() 

D:\проекты\доппроекты\GitPoster\GUI\Icons\Icon.ico
✅ Окно запущено. Не закрывай этот блок, пока используешь GUI.


0

In [5]:
# Запуск pytest и вывод результата
import subprocess
import sys
result = subprocess.run(
    [sys.executable, "-m", "pytest", "tests/unit/"],
    capture_output=True
)

# Декодируем вручную
try:
    stdout = result.stdout.decode('utf-8')
except UnicodeDecodeError:
    stdout = result.stdout.decode('cp1251', errors='replace')

try:
    stderr = result.stderr.decode('utf-8')
except UnicodeDecodeError:
    stderr = result.stderr.decode('cp1251', errors='replace')

print("🧪 Результаты тестов:\n")
print(stdout)
if stderr.strip():
    print("❌ Ошибки:\n", stderr)
if result.returncode == 0:
    print("✅ Все тесты прошли")
else:
    print("❌ Тесты провалены")

🧪 Результаты тестов:

============================= test session starts =============================
platform win32 -- Python 3.12.0, pytest-8.4.1, pluggy-1.6.0
rootdir: D:\проекты\доппроекты\GitPoster
plugins: anyio-4.9.0, mock-3.14.1
collected 4 items

tests\unit\test_file_handler.py ...                                      [ 75%]
tests\unit\test_project_manager.py .                                     [100%]

============================== 4 passed in 0.05s ==============================

✅ Все тесты прошли


## 7. Скриншоты

### Главное окно
![](docs/screenshots/image.png)

### Вкладка инициализации
![](docs/screenshots/init_tab.png)

### Логи
![](docs/screenshots/logs.png)

In [4]:
# Полезные shell-команды
print("🔧 Полезные команды:")
print("1. Создать .venv: python -m venv .venv")
print("2. Активировать: .venv\\Scripts\\activate")
print("3. Установить зависимости: pip install -r requirements.txt")
print("4. Собрать .exe: pyinstaller --onefile --windowed --icon=assets/icon.ico main.py")

🔧 Полезные команды:
1. Создать .venv: python -m venv .venv
2. Активировать: .venv\Scripts\activate
3. Установить зависимости: pip install -r requirements.txt
4. Собрать .exe: pyinstaller --onefile --windowed --icon=assets/icon.ico main.py
